# Necessary libraries

In [1]:
#!pip install pystan==2.19.1.1 convertdate==2.4.0 lunarcalendar==0.0.9 holidays==0.13 fbprophet

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS
from statistics import mean
import math
from fbprophet import Prophet
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import tensorflow as tf
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

2023-09-29 11:04:26.817062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 11:04:26.838781: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 11:04:26.839282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 11:04:27.314751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#!pip install pystan==2.19.1.1 lunarcalendar==0.0.9 holidays==0.13 fbprophet

In [3]:
df_for=pd.read_csv("6_df_interpolated_for_forecasting_aug10.csv")
df_for=df_for.drop(["Unnamed: 0"],axis=1)
df_for

,Date,Area,Gender,Race,Ethinicity,Wound
0,2016-10-26,4.0000,M,White,Non-Hispanic,1
1,2016-10-27,4.0000,M,White,Non-Hispanic,1
2,2016-10-28,4.0000,M,White,Non-Hispanic,1
3,2016-10-29,4.0000,M,White,Non-Hispanic,1
4,2016-10-30,4.0000,M,White,Non-Hispanic,1
...,...,...,...,...,...,...
1290115,2023-01-05,2.7064,F,White,Non-Hispanic,17884
1290116,2023-01-06,2.7064,F,White,Non-Hispanic,17884
1290117,2023-01-07,2.7064,F,White,Non-Hispanic,17884
1290118,2023-01-08,2.7064,F,White,Non-Hispanic,17884


In [4]:
def make_input_for_arima(df):    
    #display(df)
    df=df.reset_index()
    total_n=df.shape[0]
    n=math.ceil(0.15*total_n)
    dff=pd.concat([df, df.tail(n), df.tail(n)]).drop_duplicates(keep=False)
    #display(test)
    #display(dff)
    sample_row=dff.sample(1)
    a=int(sample_row.index.values)
    b=int(sample_row.index.values)+n
    print(a)
    print(b)
    test=df[a:b]
    train=pd.concat([df, test, test]).drop_duplicates(keep=False)
    # #display(train)
    return (test,train,a,b)

In [5]:
def model_holtwinters(df, test, train):
    #fit1 = sm.tsa.statespace.SARIMAX(train.Area, order=(2, 1, 4),seasonal_order=(0,1,1,7),initialization='approximate_diffuse').fit()
    fit1 = ExponentialSmoothing(np.asarray(train['Area']) ,seasonal_periods=2 ,trend='add', seasonal='add').fit()

    y_hat_avg = test.copy()
    y_hat_avg['Holtwinters'] = fit1.forecast(len(test))
    #print(y_hat_avg['ARIMA'])
    plt.figure(figsize=(16,8))
    #plt.plot( train['Area'], label='Train')
    #plt.plot(test['Area'], label='Test')
    plt.plot(df['Area'],label='Original')
    plt.plot(y_hat_avg['Holtwinters'], label='Forecasted by Holtwinters')
    plt.legend(loc='best')
    #plt.axhline(0)
    plt.grid()
    plt.xlabel('Days')
    plt.ylabel('Wound Area')
    plt.show()




    #Accuracy measure of the model (to be done next)
    forecast=y_hat_avg.Holtwinters
    actual=test.Area
    training_series=train.Area
    testing_series=test.Area
    prediction_series= y_hat_avg.Holtwinters



    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
     #as R-square value does not for forecasted value and actual value because forecasted values are 14 more than actual values, thus I am taking only the forecasted values minusing the last 14 values of the forecasted dataframe.
    # df = df.iloc[:-2]
    #forecast_without_last14=forecast.iloc[:-14]
    #actual_without_nan=actual.dropna()
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #corr= np.corrcoef(forecast_without_last14,actual_without_nan)[0,1]
    r_square= corr**2
    return (rmse, r_square)

In [6]:
def model_arima(test, train, df):

    # Assuming you have your DataFrame 'df' containing 'ds', 'y', and other columns
    # Sort the DataFrame by date
    df['ds'] = pd.to_datetime(df['ds'])
    df = df.sort_values('ds')

    # Initialize the AutoARIMA model
    arima_model = StatsForecast(
        models=[AutoARIMA(season_length=7)],  # You can adjust the season_length as needed
        freq='d'
    )

    # Initialize an empty DataFrame to store predictions
    predictions = []

    # Iterate over specific past dates for which you want predictions
    #specific_dates = ['2019-03-11', '2019-03-12', '2019-03-13']  # Replace with your desired dates
    specific_dates=test['ds'].values
    for date in specific_dates:
        # Filter data up to the specific date
        train_data = df[df['ds'] <= date]
        #train_data=train

        # Fit the model using the filtered data
        arima_model.fit(train_data)

        # Predict using the model for the specific date
        forecast_df = arima_model.predict(1)  # Predict one step (one day)

        # Extract the predicted y value for the specific date
        predicted_y = forecast_df['AutoARIMA'].values[0]

        # Append the prediction to the predictions DataFrame
        predictions.append({'ds': date, 'predicted_y': predicted_y})

    # Print the predictions
    #print(predictions)

    y_hat_avg = test.copy()
    y_hat_avg['ARIMA']=pd.DataFrame(predictions)['predicted_y'].values

#     print(y_hat_avg['ARIMA'])
    plt.figure(figsize=(16,8))
    #plt.plot( train['y'], label='Train')
    #plt.plot(test['y'], label='Test')
    plt.plot(df['y'],label='Original')
    plt.plot(y_hat_avg['ARIMA'], label='Forecasted by AutoARIMA')
    plt.legend(loc='best')
    #plt.axhline(0)
    plt.grid()
    plt.xlabel('Days')
    plt.ylabel('Wound Area')
    plt.show()




    #Accuracy measure of the model (to be done next)
    forecast=y_hat_avg.ARIMA
    actual=test.y
    training_series=train.y
    testing_series=test.y
    prediction_series= y_hat_avg.ARIMA



    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
     #as R-square value does not for forecasted value and actual value because forecasted values are 14 more than actual values, thus I am taking only the forecasted values minusing the last 14 values of the forecasted dataframe.
    # df = df.iloc[:-2]
    forecast_without_last14=forecast.iloc[:-14]
    actual_without_nan=actual.dropna()
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #corr= np.corrcoef(forecast_without_last14,actual_without_nan)[0,1]
    r_square= corr**2
    return (rmse, r_square)

In [7]:
def model_arima(df, test, train):
    # Assuming you have your DataFrame 'df' containing 'ds', 'y', and other columns
    # Sort the DataFrame by date
    df['ds'] = pd.to_datetime(df['ds'])
    df = df.sort_values('ds')

    # Initialize the AutoARIMA model
    arima_model = StatsForecast(
        models=[AutoARIMA(season_length=7)],  # You can adjust the season_length as needed
        freq='d'
    )

    # Initialize an empty DataFrame to store predictions
    predictions = []

    # Iterate over specific past dates for which you want predictions
    #specific_dates = ['2019-03-11', '2019-03-12', '2019-03-13']  # Replace with your desired dates
    specific_dates=test['ds'].values
    for date in specific_dates:
        # Filter data up to the specific date
        train_data = df[df['ds'] <= date]
        #train_data=train

        # Fit the model using the filtered data
        arima_model.fit(train_data)

        # Predict using the model for the specific date
        forecast_df = arima_model.predict(1)  # Predict one step (one day)

        # Extract the predicted y value for the specific date
        predicted_y = forecast_df['AutoARIMA'].values[0]

        # Append the prediction to the predictions DataFrame
        predictions.append({'ds': date, 'predicted_y': predicted_y})

    # Print the predictions
    #print(predictions)

    y_hat_avg = test.copy()
    y_hat_avg['ARIMA']=pd.DataFrame(predictions)['predicted_y'].values

    #     print(y_hat_avg['ARIMA'])
    plt.figure(figsize=(16,8))
    #plt.plot( train['y'], label='Train')
    #plt.plot(test['y'], label='Test')
    plt.plot(df['y'],label='Original')
    plt.plot(y_hat_avg['ARIMA'], label='Forecasted by AutoARIMA')
    plt.legend(loc='best')
    #plt.axhline(0)
    plt.grid()
    plt.xlabel('Days')
    plt.ylabel('Wound Area')
    plt.show()




    #Accuracy measure of the model (to be done next)
    forecast=y_hat_avg.ARIMA
    actual=test.y
    training_series=train.y
    testing_series=test.y
    prediction_series= y_hat_avg.ARIMA



    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    # df = df.iloc[:-2]
    forecast_without_last14=forecast.iloc[:-14]
    actual_without_nan=actual.dropna()
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    r_square= corr**2
    return (rmse, r_square)

In [8]:
def model_prophet(df, test, train):
    # prepare expected column names
    dff=df
    dff.columns = ['ds', 'y']
    dff['ds']= to_datetime(dff['ds'])
    # define the model
    model = Prophet(daily_seasonality=True)
    #model=Prophet()
    # fit the model
    model.fit(dff)

    #print(test.Date.iloc[0])

    # define the period for which we want a prediction
    future = []
    for j in range(0, len(test)):
        date = test.ds.iloc[j]
        future.append([date])
    future = pd.DataFrame(future)
    future.columns = ['ds']
    future['ds']= to_datetime(future['ds'])
    # use the model to make a forecast
    forecast = model.predict(future)
    
    y_hat_avg = test.copy()
    y_hat_avg['Prophet']=pd.DataFrame(forecast)['yhat'].values
    
    plt.figure(figsize=(16,8))
    #plt.plot( train['y'], label='Train')
    #plt.plot(test['y'], label='Test')
    plt.plot(df['y'],label='Original')
    plt.plot(y_hat_avg['Prophet'], label='Forecasted by Prophet')
    plt.legend(loc='best')
    #plt.axhline(0)
    plt.grid()
    plt.xlabel('Days')
    plt.ylabel('Wound Area')
    
    pyplot.show()

    #Accuracy measure of Prophet
    forecastP=forecast.yhat
    actual=test.y
    training_series=train.y
    testing_series=test.y
    prediction_series= forecast.yhat

    forecastPP=forecastP.reset_index()
    actualPP=actual.reset_index()
    training_seriesPP=training_series.reset_index()
    testing_seriesPP=testing_series.reset_index()
    prediction_seriesPP=prediction_series.reset_index()
    rmse = np.mean((forecastPP.yhat - actualPP.y)**2)**.5  # RMSE
    corr= np.corrcoef(forecastPP.yhat,actualPP.y)[0,1]     #corr
    r_square= corr**2  
    
    return (rmse, r_square)

In [9]:
def model_LSTM(df,random_number_start,random_number_end):
    # Load and preprocess your DataFrame
    data = df  # Load your DataFrame
    data['Date'] = pd.to_datetime(data['Date'])  # Convert 'Date' column to datetime
    dataframe=df.copy()
    
    # Normalize the 'Area' values
    scaler = MinMaxScaler()
    data['Area'] = scaler.fit_transform(data['Area'].values.reshape(-1, 1))

    # Create sequences and target values
    sequence_length = 1  # Adjust as needed
    sequences = []
    target_values = []

#     random_number_start=random.randint(0,data.shape[0]-int(0.20*data.shape[0]))
#     random_number_end=random_number_start+int(0.20*data.shape[0])

    data_test=data[random_number_start:random_number_end]
    remove_index=list(data_test.index.values)
    data_train=data.drop(remove_index,axis=0)

    # Make train dataset
    for i in range(len(data_train) - sequence_length):
        sequences.append(data_train['Area'].values[i:i+sequence_length])
        target_values.append(data_train['Area'].values[i+sequence_length])

    X_train = np.array(sequences)
    y_train = np.array(target_values)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    # Make test dataset
    sequences = []
    target_values = []
    for i in range(len(data_test) - sequence_length):
        sequences.append(data_test['Area'].values[i:i+sequence_length])
        target_values.append(data_test['Area'].values[i+sequence_length])

    X_test = np.array(sequences)
    y_test = np.array(target_values)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    #units=int(df.shape[0])
    
    
    model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1), return_sequences=True))
    
#     if df.shape[0]<40:
#         model.add(LSTM(50, activation='relu', return_sequences=True))
#         model.add(LSTM(50, activation='relu', return_sequences=True))
#         model.add(LSTM(50, activation='relu', return_sequences=True))
#         print("Number of total LSTM Blocks:",5)
    
#     else:
#         for k in range(0, int(df.shape[0]/20)-2):
#             model.add(LSTM(50, activation='relu', return_sequences=True))
#         print("Number of total LSTM Blocks:",k+3)

    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics=tf.metrics.MeanAbsoluteError())

    print("Total number of visit days after interpolation",df.shape[0])
    # Train the model on the training data
    model.fit(X_train, y_train, epochs=50, batch_size=16,verbose=0)

    # Make predictions on the entire dataset
    predicted_values_test = model.predict(X_test)
    predicted_values_train = model.predict(X_train)

    # Inverse transform predictions and target values
    predicted_values_test = scaler.inverse_transform(predicted_values_test)
    predicted_values_train = scaler.inverse_transform(predicted_values_train)
    y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Create arrays for plotting the entire time series
    train_dates = data_train['Date'].iloc[:-sequence_length]
    test_dates = data_test['Date'].iloc[:-sequence_length]
    all_dates=dataframe['Date'].iloc[:-sequence_length]
    all_values=dataframe['Area'].iloc[:-sequence_length]
    
    #appended_series_date = train_dates.append(test_dates, ignore_index=True)
    #appended_series_area = y_train_original.append(y_test_original, ignore_index=True)


    # Plot the entire time series data: train, test, and predicted values
    plt.figure(figsize=(16, 8))
    #plt.plot(train_dates, y_train_original, label='Train Actual')
    #plt.plot(test_dates, y_test_original, label='Test Actual')
    plt.plot(all_dates.index,all_values,label='Original')
    plt.plot(test_dates.index, predicted_values_test, label='Forecasted by LSTM')
    plt.xlabel('Days')
    plt.ylabel('Area Value')
    #plt.title('Time Series Forecasting with LSTM')
    plt.legend()
    plt.grid(True)
    plt.show()


    actual=y_test_original
    forecast=predicted_values_test
    
#     print(actual)
#     print(forecast.flatten())

    corr= np.corrcoef(actual,forecast.flatten())[0,1]
    r_square= corr**2  #r_square

    #print("R-square:",r_square)

    y_hat_avg = pd.DataFrame(y_test).copy()
    y_hat_avg['LSTM']=pd.DataFrame(forecast)[0].values


    training_series=pd.DataFrame(y_train)[0]
    testing_series=pd.DataFrame(y_test)[0]
    prediction_series= y_hat_avg.LSTM

    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return (rmse, r_square)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

def model_BiLSTM(df,random_number_start,random_number_end):
    # Load and preprocess your DataFrame
    data = df  # Load your DataFrame
    data['Date'] = pd.to_datetime(data['Date'])  # Convert 'Date' column to datetime
    dataframe=df.copy()

    # Normalize the 'Area' values
    scaler = MinMaxScaler()
    data['Area'] = scaler.fit_transform(data['Area'].values.reshape(-1, 1))

    # Create sequences and target values
    sequence_length = 1  # Adjust as needed
    sequences = []
    target_values = []

#     random_number_start = random.randint(0, data.shape[0] - int(0.20 * data.shape[0]))
#     random_number_end = random_number_start + int(0.20 * data.shape[0])

    data_test = data[random_number_start:random_number_end]
    remove_index = list(data_test.index.values)
    data_train = data.drop(remove_index, axis=0)

    # Make train dataset
    for i in range(len(data_train) - sequence_length):
        sequences.append(data_train['Area'].values[i:i + sequence_length])
        target_values.append(data_train['Area'].values[i + sequence_length])

    X_train = np.array(sequences)
    y_train = np.array(target_values)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    # Make test dataset
    sequences = []
    target_values = []
    for i in range(len(data_test) - sequence_length):
        sequences.append(data_test['Area'].values[i:i + sequence_length])
        target_values.append(data_test['Area'].values[i + sequence_length])

    X_test = np.array(sequences)
    y_test = np.array(target_values)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the BiLSTM model
    model = Sequential()
    forward_layer = LSTM(50, return_sequences=True)
    backward_layer = LSTM(50, activation='relu', return_sequences=True,go_backwards=True)
    model.add(Bidirectional(forward_layer, backward_layer=backward_layer,input_shape=(sequence_length, 1)))

    #model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(sequence_length, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    print("Total number of visit days after interpolation", df.shape[0])
    # Train the model on the training data
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

    # Make predictions on the entire dataset
    predicted_values_test = model.predict(X_test)
    #predicted_values_test=float(predicted_values_test)
    #predicted_values_train = model.predict(X_train)


    # Reshape predicted_values_test to 2D
    predicted_values_test_2d = predicted_values_test.reshape(-1, 1)

    #predicted_values_test_2d=predicted_values_test_2d.flatten()
    # Perform inverse transformation
    predicted_values_test_inverse = scaler.inverse_transform(predicted_values_test_2d)#.flatten()

    # Inverse transform predictions and target values
    #predicted_values_test = scaler.inverse_transform(predicted_values_test)
    #predicted_values_train = scaler.inverse_transform(predicted_values_train)
    y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Create arrays for plotting the entire time series
    train_dates = data_train['Date'].iloc[:-sequence_length]
    test_dates = data_test['Date'].iloc[:-sequence_length]
    all_dates=dataframe['Date'].iloc[:-sequence_length]
    all_values=dataframe['Area'].iloc[:-sequence_length]
    
    # Plot the entire time series data: train, test, and predicted values
    plt.figure(figsize=(16, 8))
    #plt.plot(train_dates, y_train_original, label='Train Actual')
    #plt.plot(test_dates, y_test_original, label='Test Actual')
    plt.plot(all_dates,all_values,label='Original')
    print("test_dates shape:" ,test_dates.shape)    
    print("predicted_values_test_inverse shape:" ,predicted_values_test_inverse.shape)
    plt.plot(test_dates, predicted_values_test_inverse, label='Forecasted by BiLSTM')
    plt.xlabel('Date')
    plt.ylabel('Area Value')
    #plt.title('Time Series Forecasting with BiLSTM')
    plt.legend()
    plt.grid(True)
    plt.show()

    actual = y_test_original
    forecast = predicted_values_test_inverse
    
    corr= np.corrcoef(actual,forecast.flatten())[0,1]
    r_square= corr**2  #r_square


    y_hat_avg = pd.DataFrame(y_test).copy()
    y_hat_avg['BiLSTM'] = pd.DataFrame(forecast)[0].values

    training_series = pd.DataFrame(y_train)[0]
    testing_series = pd.DataFrame(y_test)[0]
    prediction_series = y_hat_avg.BiLSTM

    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return rmse, r_square

## Counting the number of wounds from patients from different categories and running Prophet model

In [ ]:
list_wounds=df_for["Wound"].unique()

result_df=pd.DataFrame(index=np.arange(len(list_wounds)))

result_df["Wound"]=np.nan
result_df["Gender"]=np.nan
result_df["Race"]=np.nan
result_df["Ethinicity"]=np.nan

result_df["RMSE_Holtwinters"]=np.nan
result_df["R-squared_Holtwinters"]=np.nan

result_df["RMSE_AutoARIMA"]=np.nan
result_df["R-squared_AutoARIMA"]=np.nan

result_df["RMSE_Prophet"]=np.nan
result_df["R-squared_Prophet"]=np.nan

result_df["RMSE_LSTM"]=np.nan
result_df["R-squared_LSTM"]=np.nan

result_df["RMSE_BiLSTM"]=np.nan
result_df["R-squared_BiLSTM"]=np.nan
    

for i in range(0, len(list_wounds)):
    print("Done", i/len(list_wounds)*100,"%")
#     if i>4:
#         break
    print("Wound:",i+1)
    wound=list_wounds[i]
    df_1=df_for.query("Wound==@wound")
    #display(df_1)
    df=df_1[["Date","Area"]]
    test,train,random_number_start, random_number_end=make_input_for_arima(df)


    result_df["Wound"][i]=int(df_1["Wound"].unique()[0])
    result_df["Gender"][i]=str(df_1["Gender"].unique()[0])
    result_df["Race"][i]=str(df_1["Race"].unique()[0])
    result_df["Ethinicity"][i]=str(df_1["Ethinicity"].unique()[0])

    df=df.reset_index()
    df=df.drop(["index"],axis=1)


    if df.shape[0]>9 and df.shape[0]<1200:

                #Holt winters
                rmse_h, r_square_h=model_holtwinters(df, test, train)
                result_df["RMSE_Holtwinters"][i]=rmse_h
                result_df["R-squared_Holtwinters"][i]=r_square_h
            
                #Auto ARIMA
                df = df.rename(columns={'Date': 'ds', 'Area': 'y'})
                test = test.rename(columns={'Date': 'ds', 'Area': 'y'})
                train = train.rename(columns={'Date': 'ds', 'Area': 'y'})
                test=test.drop(['index'],axis=1)
                train=train.drop(['index'],axis=1)
                df['unique_id']='unique_wound'
                test['unique_id']='unique_wound'
                train['unique_id']='unique_wound'
                df=df.reset_index()
                df=df.drop(['index'],axis=1)
                rmse_a, r_square_a=model_arima(df, test, train)
                result_df["RMSE_AutoARIMA"][i]=rmse_a
                result_df["R-squared_AutoARIMA"][i]=r_square_a
                
                #Prophet
                df=df.drop(["unique_id"],axis=1)
                test=test.drop(["unique_id"],axis=1)
                train=train.drop(["unique_id"],axis=1)
                rmse_p, r_square_p=model_prophet(df, test, train)
                result_df["RMSE_Prophet"][i]=rmse_p
                result_df["R-squared_Prophet"][i]=r_square_p
                
                #LSTM
                df=df_1[["Date","Area"]]
                df=df.reset_index()
                df=df.drop(["index"],axis=1)
                rmse_l, r_square_l=model_LSTM(df, random_number_start,random_number_end)
                result_df["RMSE_LSTM"][i]=rmse_l
                result_df["R-squared_LSTM"][i]=r_square_l
                
                #BiLSTM
                rmse_b, r_square_b=model_BiLSTM(df, random_number_start,random_number_end)
                result_df["RMSE_BiLSTM"][i]=rmse_b
                result_df["R-squared_BiLSTM"][i]=r_square_b


result_df["Wound"]=result_df["Wound"].astype(int)

In [ ]:
i

In [153]:
#result_df

In [154]:
result_df=result_df.dropna(how='all')
#result_df

In [145]:
result_df.drop(result_df.tail(1).index,inplace=True)
#result_df

In [ ]:
result_df_prev=pd.read_csv("final_updated_result.csv")
result_df_prev=result_df_prev.drop(["Unnamed: 0"],axis=1)
result_df_prev=result_df_prev.dropna(how='all')
result_df_prev

In [156]:
#result_df_prev.describe()

In [157]:
new_df=pd.concat([result_df_prev,result_df],axis=0)
#new_df

In [159]:
#new_df.to_csv("final_updated_result.csv")
new_df.to_csv("accuracy_metrics_forecasting_models.csv")

In [163]:
metrics=pd.read_csv("accuracy_metrics_forecasting_models.csv")
metrics=metrics.drop(["Unnamed: 0"],axis=1)
metrics.describe()

,Wound,RMSE_Holtwinters,R-squared_Holtwinters,RMSE_AutoARIMA,R-squared_AutoARIMA,RMSE_Prophet,R-squared_Prophet,RMSE_LSTM,R-squared_LSTM,RMSE_BiLSTM,R-squared_BiLSTM
count,14918.000000,13884.000000,12793.000000,13883.000000,12612.000000,13883.000000,12791.000000,13883.000000,11952.000000,13883.000000,11958.000000
mean,9283.828127,17.828018,0.750778,1.630747,0.869013,3.066139,0.816213,4.818144,0.921235,0.227174,0.921251
std,5099.885820,123.410727,0.309786,15.893666,0.241408,17.658721,0.281752,24.964534,0.179202,0.150649,0.178971
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4952.250000,0.271177,0.596531,0.010575,0.879066,0.025415,0.750000,0.099157,0.938369,0.117727,0.938204
50%,9515.500000,1.343350,0.904286,0.066633,0.981187,0.192221,0.973007,0.446381,0.996026,0.196492,0.995875
75%,13668.750000,6.604105,0.990844,0.357143,0.999196,1.026750,1.000000,2.060956,0.999916,0.300956,0.999998
max,17881.000000,5938.736740,1.000000,777.998658,1.000000,632.409377,1.000000,862.068176,1.000000,0.876189,1.000000
